In [3]:
# 1. System Setup for Fall Detection Application (without Telegram integration)

# Import necessary standard libraries
import os
import time
import math
import argparse
import numpy as np
import cv2
from PIL import Image

# Import PyTorch libraries
import torch
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

# Set up GPU if available
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Define constants and configuration
CONFIG = {
    'model_path': '../models/yolov7-w6-pose.pt',  # Path to the pre-trained YOLOv7-w6-pose model
    'input_size': (960, 960),           # Input size for the model (width, height)
    'confidence_threshold': 0.25,       # Confidence threshold for detection
    'iou_threshold': 0.45,              # IoU threshold for NMS
    'device': device,                   # Device to run inference on
}

# Check if the model file exists
if not os.path.exists(CONFIG['model_path']):
    print(f"Model file not found at {CONFIG['model_path']}. Please download it first.")
    print("You can download it from: https://github.com/WongKinYiu/yolov7/releases")
else:
    print(f"Model file found at {CONFIG['model_path']}")

# Function to check camera availability
def check_camera(camera_id=0):
    cap = cv2.VideoCapture(camera_id)
    if not cap.isOpened():
        print(f"Error: Camera {camera_id} is not available")
        return False
    else:
        print(f"Camera {camera_id} is available")
        cap.release()
        return True

# Check camera
camera_available = check_camera()

# Function to download YOLOv7-W6-Pose model if not present
def download_model(model_path='yolov7-w6-pose.pt'):
    # This is a placeholder. In a real implementation,
    # you would use requests or similar to download from the GitHub repo
    print("Note: In a real implementation, this function would download the model.")
    print("Please download manually from: https://github.com/WongKinYiu/yolov7/releases")
    return False

# If model doesn't exist, try to download it
if not os.path.exists(CONFIG['model_path']):
    download_successful = download_model(CONFIG['model_path'])
    if not download_successful:
        print("Warning: Could not automatically download the model.")
        print("Please download it manually before proceeding.")

print("System setup completed!")

Using device: cpu
Model file found at ../models/yolov7-w6-pose.pt
Camera 0 is available
System setup completed!


In [ ]:
# 2. Video Processing Pipeline

def preprocess_frame(frame, input_size=(960, 960)):
    """
    Preprocess a frame for input to YOLOv7-W6-Pose.
    
    Args:
        frame (numpy.ndarray): Input frame from video/camera
        input_size (tuple): Target size for model input (width, height)
        
    Returns:
        torch.Tensor: Preprocessed frame tensor ready for model input
    """
    # Convert BGR to RGB (OpenCV uses BGR by default)
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    # Create a PIL image from the numpy array
    image = Image.fromarray(rgb_frame)
    
    # Original dimensions
    orig_width, orig_height = image.size
    
    # Calculate the letterbox dimensions to maintain aspect ratio
    ratio = min(input_size[0] / orig_width, input_size[1] / orig_height)
    new_width = int(orig_width * ratio)
    new_height = int(orig_height * ratio)
    
    # Resize the image
    resized_image = image.resize((new_width, new_height), Image.LANCZOS)
    
    # Create a new image with the target size and paste the resized image
    letterboxed_image = Image.new("RGB", input_size, (114, 114, 114))
    letterboxed_image.paste(resized_image, ((input_size[0] - new_width) // 2, 
                                           (input_size[1] - new_height) // 2))
    
    # Convert to tensor and normalize
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                             std=[0.229, 0.224, 0.225])
    ])
    
    # Apply transforms
    tensor = transform(letterboxed_image)
    
    # Add batch dimension
    tensor = tensor.unsqueeze(0)
    
    return tensor, ratio, (input_size[0] - new_width) // 2, (input_size[1] - new_height) // 2


def create_video_capture(source=0):
    """
    Create a video capture object for the specified source.
    
    Args:
        source: Camera index or video file path
        
    Returns:
        cv2.VideoCapture: VideoCapture object
    """
    # Create video capture object
    cap = cv2.VideoCapture(source)
    
    # Check if camera/video opened successfully
    if not cap.isOpened():
        print(f"Error: Could not open video source {source}")
        return None
    
    # Get frame dimensions
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    
    print(f"Video source opened: {frame_width}x{frame_height} at {fps} FPS")
    
    return cap


def process_video_source(source=0, process_frame_func=None, display=True, output_file=None):
    """
    Process frames from a video source (camera or file).
    
    Args:
        source: Camera index or video file path
        process_frame_func: Function to process each frame
        display: Whether to display the output frame
        output_file: Path to save output video (if None, no saving)
        
    Returns:
        None
    """
    # Create video capture
    cap = create_video_capture(source)
    if cap is None:
        return
    
    # Get video properties
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    
    # Create video writer if output file is specified
    out = None
    if output_file is not None:
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(output_file, fourcc, fps, (frame_width, frame_height))
    
    # Process video frames
    frame_count = 0
    start_time = time.time()
    
    while True:
        # Read frame
        ret, frame = cap.read()
        
        # Break if end of video or error
        if not ret:
            break
        
        frame_count += 1
        
        # Process the frame if a processing function is provided
        if process_frame_func is not None:
            processed_frame, fall_detected = process_frame_func(frame)
        else:
            processed_frame = frame
            fall_detected = False
        
        # Write frame to output video if specified
        if out is not None:
            out.write(processed_frame)
        
        # Display the frame
        if display:
            # Add fall detection status to the frame
            if fall_detected:
                cv2.putText(processed_frame, "FALL DETECTED", (50, 50), 
                           cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            
            cv2.imshow('Video Processing', processed_frame)
            
            # Break if 'q' is pressed
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
    
    # Calculate and print processing stats
    elapsed_time = time.time() - start_time
    if frame_count > 0 and elapsed_time > 0:
        fps_processing = frame_count / elapsed_time
        print(f"Processed {frame_count} frames in {elapsed_time:.2f} seconds ({fps_processing:.2f} FPS)")
    
    # Release resources
    cap.release()
    if out is not None:
        out.release()
    cv2.destroyAllWindows()

# Test the video processing pipeline
if __name__ == "__main__":
    # This will just display the camera feed without any processing
    process_video_source(0, None, True, None)

In [ ]:
# 4. Fall Detection Algorithm - Including Fall vs. Lying Down Differentiation

class FallDetector:
    def __init__(self):
        """Initialize the fall detector with tracking for speed calculation."""
        self.prev_keypoints = None
    
    def calculate_length_factor(self, keypoints):
        """
        Calculate length factor based on the shoulder-to-torso distance.
        
        This implements Equation 1 from page 9 of the journal:
        Lfactor = √(xl - xTl)² + (yl - yTl)²
        
        Args:
            keypoints (list): List of keypoint tuples (x, y, confidence)
            
        Returns:
            float: Length factor for distance calculations
        """
        # Get left shoulder and left hip (torso) keypoints
        left_shoulder = keypoints[KEYPOINT_DICT['left_shoulder']]
        left_hip = keypoints[KEYPOINT_DICT['left_hip']]
        
        # Calculate Euclidean distance as in Equation 1
        shoulder_x, shoulder_y = left_shoulder[0], left_shoulder[1]
        hip_x, hip_y = left_hip[0], left_hip[1]
        
        length_factor = math.sqrt((shoulder_x - hip_x)**2 + (shoulder_y - hip_y)**2)
        return length_factor
    
    def calculate_vertical_speed(self, prev_keypoints, curr_keypoints):
        """
        Calculate vertical speed of movement to differentiate falls from lying down.
        
        As described on page 10: "The speed of key body points is calculated by
        measuring the displacement between their positions in consecutive frames."
        
        Args:
            prev_keypoints (list): Previous frame keypoints
            curr_keypoints (list): Current frame keypoints
            
        Returns:
            float: Vertical speed (displacement between frames)
        """
        if prev_keypoints is None:
            return 0
        
        # Use shoulders to calculate vertical speed as mentioned in the paper
        prev_left_shoulder = prev_keypoints[KEYPOINT_DICT['left_shoulder']]
        curr_left_shoulder = curr_keypoints[KEYPOINT_DICT['left_shoulder']]
        
        # Calculate vertical displacement
        vertical_displacement = abs(curr_left_shoulder[1] - prev_left_shoulder[1])
        return vertical_displacement
    
    def calculate_torso_angle(self, keypoints):
        """
        Calculate angle between torso and vertical.
        
        As mentioned on page 10: "A threshold of 45 degrees is used in the code.
        If the angle between the torso and legs drops below this value, it indicates
        that the person's body is approaching a horizontal position..."
        
        Args:
            keypoints (list): List of keypoint tuples (x, y, confidence)
            
        Returns:
            float: Angle in degrees
        """
        # Get shoulder and hip keypoints to define torso
        left_shoulder = keypoints[KEYPOINT_DICT['left_shoulder']]
        right_shoulder = keypoints[KEYPOINT_DICT['right_shoulder']]
        left_hip = keypoints[KEYPOINT_DICT['left_hip']]
        right_hip = keypoints[KEYPOINT_DICT['right_hip']]
        
        # Calculate midpoints
        mid_shoulder_x = (left_shoulder[0] + right_shoulder[0]) / 2
        mid_shoulder_y = (left_shoulder[1] + right_shoulder[1]) / 2
        mid_hip_x = (left_hip[0] + right_hip[0]) / 2
        mid_hip_y = (left_hip[1] + right_hip[1]) / 2
        
        # Calculate torso vector
        torso_x = mid_shoulder_x - mid_hip_x
        torso_y = mid_shoulder_y - mid_hip_y
        
        # Calculate angle with vertical (y-axis in image coordinates)
        # Note: In image coordinates, y increases downward
        angle_rad = math.atan2(torso_x, torso_y)  # Angle with vertical
        angle_deg = math.degrees(abs(angle_rad))
        
        return angle_deg
    
    def detect_fall(self, people):
        """
        Detect if a fall has occurred based on pose keypoints.
        
        This implements the fall detection algorithm described in the paper
        on pages 9-10, including the differentiation between falls and lying down.
        
        Args:
            people (list): List of dictionaries containing keypoints and bounding boxes
            
        Returns:
            bool: True if fall detected, False otherwise
        """
        # If no people detected, return False
        if not people:
            self.prev_keypoints = None
            return False
        
        # Use the first person detected
        person = people[0]
        keypoints = person['keypoints']
        
        # Get relevant keypoints
        left_shoulder = keypoints[KEYPOINT_DICT['left_shoulder']]
        right_shoulder = keypoints[KEYPOINT_DICT['right_shoulder']]
        left_hip = keypoints[KEYPOINT_DICT['left_hip']]
        right_hip = keypoints[KEYPOINT_DICT['right_hip']]
        left_ankle = keypoints[KEYPOINT_DICT['left_ankle']]
        right_ankle = keypoints[KEYPOINT_DICT['right_ankle']]
        
        # 1. Calculate the length factor (Equation 1, page 9)
        length_factor = self.calculate_length_factor(keypoints)
        
        # 2. Check if shoulders are lower than feet with adjustment (Equation 2, page 9)
        # yl ≤ yFl + α·Lfactor
        # Note: In image coordinates, y increases downward, so we flip the inequality
        alpha = 0.1  # Small adjustment factor as mentioned in the paper
        shoulder_below_feet = (
            left_shoulder[1] >= left_ankle[1] - alpha * length_factor or
            right_shoulder[1] >= right_ankle[1] - alpha * length_factor
        )
        
        # 3. Calculate body height and width (Equations 3 & 4, page 10)
        # Hbody = |yl - yFl|
        body_height = abs(left_shoulder[1] - left_ankle[1])
        # Wbody = |xl - xr|
        body_width = abs(left_shoulder[0] - right_shoulder[0])
        
        # 4. Check fall condition based on body dimensions (Equation 5, page 10)
        # Hbody < Wbody
        orientation_fallen = body_height < body_width
        
        # 5. Differentiate between fall and lying down (as described on page 10)
        # Calculate vertical speed between frames
        vertical_speed = 0
        if self.prev_keypoints is not None:
            vertical_speed = self.calculate_vertical_speed(self.prev_keypoints, keypoints)
        
        # Calculate torso angle
        torso_angle = self.calculate_torso_angle(keypoints)
        
        # From page 10: "If the vertical speed exceeds a specific threshold, it indicates a fall..."
        # From page 10: "A threshold of 45 degrees is used in the code."
        is_rapid_movement = vertical_speed > 15  # Threshold for rapid movement
        is_horizontal_position = torso_angle > 45  # Angle threshold as mentioned in paper
        
        # Fall is detected if basic conditions are met AND
        # either the movement is rapid OR the body is in a horizontal position
        fall_detected = (
            shoulder_below_feet and 
            orientation_fallen and 
            (is_rapid_movement or is_horizontal_position)
        )
        
        # Store current keypoints for next frame's speed calculation
        self.prev_keypoints = keypoints
        
        return fall_detected
    
    def annotate_frame(self, frame, fall_detected):
        """
        Annotate the frame with fall detection status.
        
        Args:
            frame (numpy.ndarray): Frame to annotate
            fall_detected (bool): Whether a fall is detected
            
        Returns:
            numpy.ndarray: Annotated frame
        """
        if fall_detected:
            # Draw red text for fall detection
            cv2.putText(
                frame,
                "FALL DETECTED!",
                (50, 50),
                cv2.FONT_HERSHEY_SIMPLEX,
                1,
                (0, 0, 255),  # Red color
                2
            )
        
        return frame

# Function to process a frame with fall detection
def process_frame_with_fall_detection(frame, model, device, fall_detector):
    """
    Process a frame with pose estimation and fall detection.
    
    Args:
        frame (numpy.ndarray): Input frame
        model: YOLOv7-W6-Pose model
        device (torch.device): Device to run inference on
        fall_detector (FallDetector): Fall detector instance
        
    Returns:
        tuple: (Processed frame with annotations, fall detection status)
    """
    # Perform pose estimation
    frame_with_pose, people = process_frame_with_pose(frame, model, device)
    
    # Detect falls using the algorithm from the journal
    fall_detected = fall_detector.detect_fall(people)
    
    # Annotate frame with fall detection status
    annotated_frame = fall_detector.annotate_frame(frame_with_pose, fall_detected)
    
    return annotated_frame, fall_detected

# Test fall detection if running this cell directly
if __name__ == "__main__":
    # Only import if this file is run directly
    from video_processing import process_video_source
    
    # Load the model
    model = load_model(CONFIG['model_path'], CONFIG['device'])
    
    # Create fall detector
    fall_detector = FallDetector()
    
    # Define a function to process each frame with the model and fall detection
    def process_frame_func(frame):
        return process_frame_with_fall_detection(frame, model, device, fall_detector)
    
    # Process video from webcam
    process_video_source(0, process_frame_func, True, None)

In [ ]:
# 5. Main Program Integration

def main():
    """
    Main function that integrates all components of the fall detection system.
    """
    print("Fall Detection System using YOLOv7-W6-Pose")
    print("------------------------------------------")
    
    # Parse command line arguments
    parser = argparse.ArgumentParser(description='Fall Detection System using YOLOv7-W6-Pose')
    parser.add_argument('--source', type=str, default='0', 
                        help='Source for video input (0 for webcam, or path to video file)')
    parser.add_argument('--output', type=str, default=None, 
                        help='Path to save processed video (None for no saving)')
    parser.add_argument('--display', action='store_true', 
                        help='Display video processing in real-time')
    args = parser.parse_args()
    
    # Convert source to int if it's a digit (camera index)
    if args.source.isdigit():
        args.source = int(args.source)
    
    # Check if model file exists
    if not os.path.exists(CONFIG['model_path']):
        print(f"Error: Model file not found at {CONFIG['model_path']}")
        print("Please download it from: https://github.com/WongKinYiu/yolov7/releases")
        return
    
    try:
        # Load YOLOv7-W6-Pose model
        print(f"Loading model from {CONFIG['model_path']}...")
        model = load_model(CONFIG['model_path'], CONFIG['device'])
        print("Model loaded successfully.")
        
        # Create fall detector
        fall_detector = FallDetector()
        
        # Define frame processing function
        def process_frame_func(frame):
            return process_frame_with_fall_detection(frame, model, CONFIG['device'], fall_detector)
        
        # Process video source
        print(f"Processing video from source: {args.source}")
        if args.output:
            print(f"Output will be saved to: {args.output}")
        
        process_video_source(args.source, process_frame_func, args.display, args.output)
        
        print("Processing completed.")
        
    except Exception as e:
        print(f"Error: {str(e)}")
        import traceback
        traceback.print_exc()

# Run the main program if this script is executed directly
if __name__ == "__main__":
    main()